In [18]:
import os

In [19]:
patent_abstract = """
A kind of independent vehicular modular wheel set and rear-axle steering control method driven with turning to
Abstract
The invention discloses a kind of for four motorized wheels and the electromobile modularization wheel set of steering, including vehicle frame；Steering motor is fixed on the vehicle frame；And subframe, form accommodation space；The subframe is rotatably supported on the vehicle frame；Subframe described in the output axis connection of the steering motor, the subframe can be rotated around output shaft；Driving motor is fixed in the accommodation space, for wheel to be driven to rotate；Column, wheel hub are rotatably supported on the column；The column has through hole, and the output shaft of the driving motor drives wheel through the through hole；Upper suspension swing arm, both ends are connected to by rubber packing between the upper end of the accommodation space and the column；Lower suspension swing arm, both ends are connected to by rubber packing between the lower end of the accommodation space and the column.Wheel set integrated driving, braking, suspension and the steering of the present invention realizes four motorized wheels with turning to.
"""


In [20]:
from openai import OpenAI
oai_client = OpenAI()

oai_client.embeddings.create(
        model="text-embedding-ada-002",
        input=patent_abstract
    )


CreateEmbeddingResponse(data=[Embedding(embedding=[-0.0021371457260102034, 0.011004604399204254, -0.024980859830975533, -0.012164769694209099, -0.02499442920088768, 0.022809790447354317, -0.015821663662791252, -0.011452388018369675, -0.004433729685842991, -0.009681609459221363, -0.003288829931989312, 0.02029949240386486, -0.012286892160773277, 0.008813181892037392, -0.01060431357473135, 0.008914951235055923, 0.026093533262610435, 0.01906469836831093, 0.015685971826314926, -0.02963508851826191, 0.0011601648293435574, -0.005817785859107971, -8.703356434125453e-05, -0.0009617156465537846, -0.008996366523206234, 0.0011856070486828685, 0.02510298229753971, -0.009735886938869953, 0.012795737013220787, -0.013168889097869396, 0.013921978883445263, 0.005397141445428133, 0.007714078761637211, -0.01736854948103428, -0.0017419434152543545, -0.008921735920011997, 0.03063920885324478, -0.015116066671907902, 0.0054310644045472145, 0.01740925759077072, 0.03454713150858879, 0.007415556814521551, -0.015

In [21]:
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

embedding_function = OpenAIEmbeddingFunction(api_key=os.environ.get('OPENAI_API_KEY'),
                                             model_name="text-embedding-ada-002")


chroma_client = chromadb.Client()
vector_store = chroma_client.get_or_create_collection(name="Patents",
                                                      embedding_function=embedding_function)

In [22]:
vector_store.add("patent_info", documents=patent_abstract)

Add of existing embedding ID: patent_info
Insert of existing embedding ID: patent_info


In [23]:
from trulens_eval import Tru
from trulens_eval.tru_custom_app import instrument
tru = Tru()
#tru.reset_database() #Reset the database. Clears all tables.


In [24]:
class RAG_from_scratch:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        results = vector_store.query(
        query_texts=query,
        n_results=2
    )
        return results['documents'][0]

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        completion = oai_client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=
        [
            {"role": "user",
            "content": 
            f"The following abstract descripes a concept for a novel invention: \n"
            f"---------------------\n"
            f"{context_str}"
            f"\n---------------------\n"
            f"Given this information, please execute the following instructions: {query}"
            }
        ]
        ).choices[0].message.content
        return completion

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve(query)
        completion = self.generate_completion(query, context_str)
        return completion

rag = RAG_from_scratch()

In [25]:
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI as fOpenAI

import numpy as np

# Initialize provider class
fopenai = fOpenAI()

grounded = Groundedness(groundedness_provider=fopenai)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = (
    Feedback(fopenai.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(fopenai.qs_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.app.retrieve.args.query .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.app.retrieve.args.query .
✅ In Context Relevance, input statement will be set to __record__.app.retrieve.rets.collect() .


In [26]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(rag,
    app_id = 'RAG Key Words',
    feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

Function <function RAG_from_scratch.generate_completion at 0x0000026262D8C280> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x0000026264A77BE0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG_from_scratch.query at 0x0000026262D8C310> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x0000026264A77BE0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG_from_scratch.retrieve at 0x0000026262D8C1F0> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x0000026264A77BE0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.


In [27]:
with tru_rag as recording:
    rag.query("Name 5 key words based on this abstract, that I can use for the search in a patent database. Optimize the key words to get back more results. Result as python string.")

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


In [28]:
tru.get_leaderboard(app_ids=["RAG Key Words"])

,Context Relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,
RAG Key Words,0.9,1.0,1.0,3.5,0.0


In [29]:

tru_rag = TruCustomApp(rag,
    app_id = 'RAG Classifications',
    feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

Function <function RAG_from_scratch.generate_completion at 0x0000026262D8C280> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x0000026264A77BE0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG_from_scratch.query at 0x0000026262D8C310> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x0000026264A77BE0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG_from_scratch.retrieve at 0x0000026262D8C1F0> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x0000026264A77BE0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.


In [30]:
with tru_rag as recording:
    rag.query("Name 5 CPC classifications based on this abstract, that I can use for the search in a patent database. \
Please give me a python string for the codes of the 5 most relevant \
CPC classifications to a possible patent.")

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


In [31]:
tru.get_leaderboard(app_ids=["RAG Classifications"])

,Context Relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,
RAG Classifications,0.466667,0.14,1.0,3.75,0.0


In [32]:

tru_rag = TruCustomApp(rag,
    app_id = 'RAG Patent Comparison',
    feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

Function <function RAG_from_scratch.generate_completion at 0x0000026262D8C280> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x0000026264A77BE0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG_from_scratch.query at 0x0000026262D8C310> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x0000026264A77BE0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG_from_scratch.retrieve at 0x0000026262D8C1F0> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x0000026264A77BE0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.


In [33]:
with tru_rag as recording:
    rag.query(f"""
'The following texts are abstracts from patent specifications. Your task is to compare the novel invention from above to all the other abstracts from below. \n
It is important that you focus on comparing the concepts that the abstracts describe, not the way they are written. \n
Rank the remaining abstracts on how well they match with the Testing Abstract by giving them a rating from 0 to 10 points. \n        
0 meaning they have absolutely nothing in common and 10 meaning they basically describe the exact same idea.\n  
Your output should be a python dictionary with the title "comparison", each element hast the Abstract number as key and the rating as value.\n        
I want to convert your output string to an actual dictionary, so make sure the formatting is right.\n\n        
        
CN105882741B: "The invention discloses a kind of for four motorized wheels and the electromobile modularization wheel set of steering, including vehicle frame；Steering motor is fixed on the vehicle frame；And subframe, form accommodation space；The subframe is rotatably supported on the vehicle frame；Subframe described in the output axis connection of the steering motor, the subframe can be rotated around output shaft；Driving motor is fixed in the accommodation space, for wheel to be driven to rotate；Column, wheel hub are rotatably supported on the column；The column has through hole, and the output shaft of the driving motor drives wheel through the through hole；Upper suspension swing arm, both ends are connected to by rubber packing between the upper end of the accommodation space and the column；Lower suspension swing arm, both ends are connected to by rubber packing between the lower end of the accommodation space and the column.Wheel set integrated driving, braking, suspension and the steering of the present invention realizes four motorized wheels with turning to."\nUS20060237242A1: "Lightweight wheeled surface vehicles of various types and sizes constructed chiefly from commercial off-the-shelf (COTS) parts, incorporate alternate suspensions, e.g. swingarms. One embodiment provides a vehicle incorporating a cellular body design wherein the vehicle is constructed from a varying number of substantially identical cells, assembled end-to-end to produce vehicles of varying size and capacity. Additional embodiments include lightweight passenger vehicles, such as automobiles, manufacturable from COTS parts, including independent suspensions providing large vertical wheel travel. One embodiment provides an automobile-type vehicle having a roll-cage frame, and a lightweight, exo-skeleton external frame, provided in multiple wheel configurations, e.g. three- or four-wheeled configurations. Body panels are quickly and easily attached to the tubular frame and also easily removed and switched and readily replaceable. Bicycles are equipped with electric pedal assist units. Additionally, a pneumatic pedal assist reduces peak power requirements and prolongs battery life. "\nUS20090091101A1: "A vehicle (100) is disclosed. The vehicle may include a base portion (104) and a modular portion (112). The base portion may be a four wheel vehicle having an operator area (114) with seating for at least two occupants in a side-by-side arrangement. The modular portion may be coupled to the base portion resulting in a six wheel vehicle. The wheels (102) of the modular portion may be powered by an engine (500) of the base portion."\nUS5343974A: "A motor vehicle including modular units for operating two rear wheels of the vehicle the motive unit includes a rigid link which is pivotally connected between the rear wheel and the motive unit, and the motive unit is operable by a motor. A rigid link is pivotally connected, and the rear wheels are spring suspended mounted relative to the frame of the vehicle. In an electrical vehicle form, there is a slidable movable battery pack located in a rear compartment of the vehicle for easy installation and removal from the vehicle. Different traction characteristics can be provided to the motive unit including an integrated motor, integrated gear box, and power input-output shaft accessible to secondary power or an energy recycler unit. Steering is effected by a controller which operates the speed of the driven wheels."\nUS5934397A: "A modular vehicle includes a frame having three main sections; a front section, a rear section and a center section. The front and rear sections may be displaced vertically with respect to center section prior to being assembled. Each section is separately assembled and standardized so that various devices may be incorporated to each section and each section may be otherwise modified to allow a specific vehicle to be easily and quickly manufactured."\nUS20190039651A1: "A method for influencing the direction of travel of motor vehicles. The motor vehicle has a first steering system, having at least one steering axle with at least two wheels. The wheels are connected to the steering axle through wheel suspensions that can be adjusted by actuators. A redundant steering system is realized by adjusting a steering roll radius of at least one of the steering axles, according to which a force component acting orthogonally to the direction of travel in the region of at least one wheel is applied to the first steering system."\nCN115003586A: "The invention relates to a method for controlling a steer-by-wire steering system of a motor vehicle (1), wherein the motor vehicle (1) comprises two axles (10, 20) each having two wheels (RL, RR, FL, FR), wherein the two front wheels (FL, FR) can be steered by means of front wheel steering and the two rear wheels (RL, RR) can be steered by means of rear wheel steering, and the motor vehicle (1) comprises a steering system with two axles (10, 20)20) and via a differential to drive two wheels of the corresponding axle, wherein the motor vehicle (1) comprises an on-board braking system, and the method comprises the steps of: checking the speed of the motor vehicle; enabling rear axle steering if the motor vehicle speed is less than 40 km/hr; when the steering of the rear axle is in the enabled state, the following steps are implemented: deactivating front wheel steering and rear wheel steering; by means of a set point wheel steering angle (alpha) RW,ref ) To determine the set point position of the first steering column (S) R,ref ) (ii) a Determining a differential drive torque (Δ T) between the two rear wheels (RL, RR) by means of a control unit in order to reach the reference position (S) R,ref )。"\nCN111836745B: "The invention relates to a steering method for an autonomously steered vehicle having a hybrid steering system with a hydraulic steering support device, an electromechanical steering support device and a control device for monitoring and controlling the driving process of the vehicle. The method comprises the following method steps: identifying a functional fault during an autonomous driving procedure; the control unit switches the steering to a steering brake, in which the steering is performed by braking at least one wheel, so that a steering torque results from the braking force acting at the steering radius, which steering torque leads to a deflection of the wheel. The invention also relates to a servo steering assembly enabling the method of the invention to be carried out."\nCN106080263B: "The invention discloses a kind of electric wheel truck chassis system and its optimization method, chassis system includes differential steering module, differential braking module and Active suspension module；Differential steering module includes steering wheel torque rotary angle transmitter, rack and pinion steering gear, two wheel hub motors, vehicle speed sensor, two wheel speed sensors, yaw-rate sensor and differential steering control ECU；Active suspension module includes flexible member, damping element, forcer and guiding mechanism；Differential braking module includes brake pedal position sensor and differential braking control ECU.Under steering/damped condition, using steering response, steering sensitivity, suspension ride comfort as performance indications, using the parameter of three modules of the invention as optimized variable, chassis system is optimized based on the algorithms of NSGA II, system is set to obtain preferable steering response and steering sensitivity while ensure automobile ride, so as to improve the whole synthesis performance of electric wheel truck chassis system."\nUS7338335B1: "A hybrid electric system designed to lower fuel consumption in heavy-duty long-haul vehicles, and in medium and light duty vehicles (trucks, buses, vans, SUVs, recreational vehicles, and the like), utilizing a multiplicity of thermal engines, regenerative power road-wheels, solar cells, and frontal area reducing adjustable-height suspension that are utilized singly or in combinations as suits the vehicle\'s mission."\nSE1951159A1: "The present invention relates to vehicle steering system arrangement (100) comprising a steerable vehicle suspension and power assisted vehicle steering with a feedback torque actuator (130), the steering feel being controlled by means of a torque and/or angle control system, TAC, (132). The steerable vehicle suspension comprises a plurality of vehicle suspension parameters and the geometry and/or dimensions of one or more of the vehicle suspension parameters or elements is/are designed to reduce or minimize the steering force required for steering the vehicle in driving. The torque and/or angle control, TAC, (132) is used for, based at least on directly or indirectly sensed and/or calculated steering wheel angle, controlling the wheel steer angle and providing feedback control by controlling the feedback torque actuator to generate a target feedback torque, and a target steering feel, and hence reducing or eliminating driver feedback provided by the vehicle suspension, such that the steering effort, if there is a fault in the steering control so thet the assistance is lost, will be low enough for the driver to handle safely, hence without needing failoperational steering gear."\nUS20190152513A1: "A steer-by-wire system for a motor vehicle includes at least two wheels which are steerable independently of one another in a normal operating mode of the steer-by-wire system, at least two steering actuators, each one being assigned to one of the steerable wheels and being configured for adjusting a steering angle of the particular steerable wheel, and at least one steering electronics system which is signally connected to the steering actuators and which is configured for controlling the steering actuators individually on the basis of steering commands."\nAU2009295555B2: "A pneumatic vehicle is provided with a first sub- assembly with a chassis, part of the vehicle body, a pair of B-pillars, a pair of rear rails, wheels, an elongate aluminum compressed load bearing air tank oriented longitudinally in the chassis, side panels connected to the tank and the wheels, a heat exchanger to heat the compressed air, and an air motor driven by the heated, compressed air and connected to a wheel. A ventilation system has a restrictive solenoid valve for directing air to the heat exchanger. The air tank is provided with a carbon filament reinforced plastic layer, and a fiberglass and aramid-fiber layer. A second sub-assembly includes part of the vehicle body bonded to the first-sub-assembly using a structural adhesive, a pair of A-pillars, and a pair of roof rails. Seating includes inflatable components for adjustment."\nKR100421832B1: "A modular track suspension system is readily adapted for attachment to any presently manufactured full-size automotive vehicle built with a load-carrying frame, being easily substituted for the vehicle\'s conventional wheeled undercarriage either during or following final assembly. The module uses a pair of endless rubber tracks, one track under each respective side of the vehicle; and each track is frictionally driven by tandem pairs of dual-wheels with rubber tires, each tandem pair of wheels being driven, respectively, by an intermediately positioned drive-unit axle. The dual-wheels are each movable in a vertical plane, and each dual-wheel can move separately and independently of the similar independent movement of the other dual-wheel with which it is tandemly paired. Each dual-wheel is also resiliently biased in the direction of the terrain on which the vehicle is standing or moving. Driving torque is delivered to the two endless tracks through a plurality of differentials appropriately and simply connected directly with the vehicle\'s drive shaft. In all preferred embodiments, an additional differential that is connected with, and is responsive to, the operation of the vehicle\'s steering wheel superimposes additive and subtractive steering torques to the already differentiated driving torques being delivered to the left-side and right-side tracks for controlling the vehicle\'s direction."\nCN114929563A: "The invention relates to a vehicle steering system (10) having: -at least one servomotor (18), -a steering handle (12) by means of which a desired driver steering angle (FW) can be preset by the driver; -a controller (24) having a driver assistance function (32) with which an auxiliary steering Angle (AW) for actuating the servomotor (18) can be preset autonomously of the driver; and the control device has at least one further function (34) with which a restoring torque can be preset according to the obtained angular difference in order to reduce the angular difference; wherein, if the driver specifies the driver steering angle (FW) with the driver assistance function (32) activated, a restoring torque can be preset by means of a further function (34) on the basis of the angular difference between the driver steering angle (FW) and the assistance steering Angle (AW). The invention further relates to a method for operating a vehicle steering system (10) and to a vehicle having a vehicle steering system (10)."\nUS10858040B2: "Methods for controlling a feedback torque actuator and at least one yaw and/or lateral vehicle state actuator in a steer-by-wire steering system include measuring an input signal with a sensor, determining from the input signal a measure of a torque applied by the driver via a steering wheel, transforming the measure to a desired yaw and/or lateral vehicle state, controlling the yaw and/or lateral vehicle state actuator for vehicle state control, and defining a steering-wheel torque to steering-wheel angle relation describing steering feel. If the vehicle position control results in a yaw and/or lateral vehicle state error, this error is transformed to a change in the steering-wheel torque to steering-wheel angle relation describing steering feel. This new steering feel relation is used as an input signal for controlling the feedback torque actuator in order for the driver to get feedback of the yaw and/or lateral vehicle state error."\nUS20100269920A1: "The RABBAT ELECTRIC HYBRID VEHICLES have no mechanical transmission of power, only ductile wires connecting various sections of the car."\n'
""")

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


In [34]:
tru.get_leaderboard(app_ids=["RAG Patent Comparison"])

,Context Relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,
RAG Patent Comparison,0.533333,0.888889,0.533333,11.25,0.0
